In [1]:
import torch
import numpy as np
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline
from torch import cuda
import pandas as pd
import numpy as np
import torch
import transformers
import json
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaModel, RobertaTokenizer
from torch.optim import Adam
from torch.nn import CrossEntropyLoss
import logging
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

device = 'cuda' if cuda.is_available() else 'cpu'


/home/kaie/miniconda3/envs/discord/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class RobertaClass(torch.nn.Module):
    def __init__(self):
        super(RobertaClass, self).__init__()
        self.l1 = RobertaModel.from_pretrained("roberta-base")
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.3)
        self.classifier = torch.nn.Linear(768, 50266)

    def forward(self, input_ids, attention_mask, token_type_ids):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output

In [5]:
model = RobertaClass()
model = torch.load('./model_weights.pth')
tokenizer = AutoTokenizer.from_pretrained('./CTM{1_4m_4w3$0m3!1_kn0w_h0w_70_d1ff3r3n71473_r34l_@nd_f@k3!}')
model.eval() 


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaClass(
  (l1): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((

In [59]:
input = tokenizer("This is an AI generated because the duck's neck, building's windows, and the clouds are weird" , "this is ai. The duck's neck is wrong, blah blah blah blah building's windows are blurred, and the clouds are not right. ", return_tensors='pt', return_token_type_ids=True )
token_type_ids = input['token_type_ids'].to(device)
mask = input['attention_mask'].to(device)
tokens = input['input_ids'].to(device)
print(tokenizer.decode(torch.argmax(model(tokens, mask, token_type_ids))))

CTM{1_4m_4w3$0m3!1_kn0w_h0w_70_d1ff3r3n71473_r34l_@nd_f@k3!}
